### Detach one from another

In [1]:
import trimesh
import numpy as np
from utils import *

In [2]:
mesh = trimesh.load("data/two_spheres2.ply")

Split the mesh into connected components

In [3]:
connected_components = mesh.split(only_watertight=True)

Compute centroids of each component

In [4]:
centroids = [comp.centroid for comp in connected_components]

Compute the vector between the centroids

In [5]:
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance

Shift the second component along the normalized vector

In [ ]:
connected_components[1].apply_translation(normalized_vector * 0.7)

<trimesh.Trimesh(vertices.shape=(476, 3), faces.shape=(948, 3), name=`two_spheres2.ply`)>

In [7]:
new_mesh = trimesh.util.concatenate(connected_components)

In [8]:
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)

In [9]:
visualize_two_meshes(mesh, x)

Widget(value='<iframe src="http://localhost:37609/index.html?ui=P_0x7f3b9e3a10f0_0&reconnect=auto" class="pyvi…

None

In [1]:
import trimesh
import numpy as np
from utils import *

In [38]:
mesh = trimesh.load("data/shrink.ply")
connected_components = mesh.split(only_watertight=True)
centroids = [comp.centroid for comp in connected_components]
translation_vector = centroids[1] - centroids[0]
distance = np.linalg.norm(translation_vector)
normalized_vector = translation_vector / distance
connected_components[1].apply_translation(normalized_vector * 0.079)
new_mesh = trimesh.util.concatenate(connected_components)
x = pymesh.form_mesh(vertices=new_mesh.vertices, faces = new_mesh.faces)
print(pymesh.detect_self_intersection(x))

[[ 465  844]
 [ 465 1199]
 [ 465  830]
 [ 465 1156]
 [ 828  465]
 [1154  465]
 [1211  465]
 [ 864  465]
 [1232  465]
 [ 825  465]
 [1194  465]]


In [39]:
visualize_intersection(x)

Widget(value='<iframe src="http://localhost:40761/index.html?ui=P_0x7f2247f8a320_17&reconnect=auto" class="pyv…

None

In [41]:
pymesh.save_mesh("data/shrink.ply", x)

In [11]:
import trimesh
import open3d as o3d
import numpy as np

def trimesh_to_open3d(tri_mesh: trimesh.Trimesh) -> o3d.geometry.TriangleMesh:
    """
    Convert a trimesh.Trimesh to an open3d.geometry.TriangleMesh.
    """
    o3d_mesh = o3d.geometry.TriangleMesh()
    o3d_mesh.vertices = o3d.utility.Vector3dVector(tri_mesh.vertices)
    o3d_mesh.triangles = o3d.utility.Vector3iVector(tri_mesh.faces)
    return o3d_mesh


def detach_inner_sphere_trimesh(
    input_path,
    output_path,
    contact_threshold=0.001,
    num_sample_points=50000
):
    """
    1) Use trimesh to read and split (connected components).
    2) Convert each submesh to Open3D geometry.
    3) Push the inner mesh inward where it contacts the outer mesh.
    4) (Optionally) smooth the inner mesh.
    5) Save final mesh in Open3D.
    """
    # 1) Read mesh with trimesh
    tm = trimesh.load(input_path, process=False)
    
    # 2) Split into connected components (sub-meshes)
    submeshes = tm.split(only_watertight=True) 
    
    # Sort by number of faces  ### FIX THIS!!!!
    submeshes_sorted = sorted(submeshes, key=lambda m: m.volume)
    mesh_inner_tm = submeshes_sorted[0]
    mesh_outer_tm = submeshes_sorted[-1]
    
    # 3) Convert to Open3D
    mesh_inner = trimesh_to_open3d(mesh_inner_tm)
    mesh_outer = trimesh_to_open3d(mesh_outer_tm)
    
    mesh_inner.compute_vertex_normals()
    mesh_outer.compute_vertex_normals()

    # Sample outer for distance checks
    pcd_outer = mesh_outer.sample_points_poisson_disk(number_of_points=num_sample_points)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_outer)

    inner_vertices = np.asarray(mesh_inner.vertices)
    center_inner = np.mean(inner_vertices, axis=0)

    # For each vertex, find distance to outer sphere
    for i in range(len(inner_vertices)):
        v = inner_vertices[i]
        k, idx, dist_sq = pcd_tree.search_knn_vector_3d(v, 1)
        if k > 0:
            closest_point = np.asarray(pcd_outer.points)[idx[0]]
            dist = np.linalg.norm(v - closest_point)
            if dist < contact_threshold:
                # shrink radially inward by (contact_threshold - dist)
                offset = contact_threshold - dist
                direction = v - center_inner
                r_len = np.linalg.norm(direction)
                if r_len > 1e-12:
                    direction_unit = direction / r_len
                    inner_vertices[i] = v - offset * direction_unit


    mesh_inner.vertices = o3d.utility.Vector3dVector(inner_vertices)


    # 7) Merge the two meshes in Open3D
    combined_mesh = mesh_inner + mesh_outer
    
    # 8) Write out the combined mesh
    o3d.io.write_triangle_mesh(output_path, combined_mesh)
    print(f"Output saved to {output_path}")



input_ply = "data/shrink.ply"
output_ply = "temp.ply"
detach_inner_sphere_trimesh(
    input_path=input_ply,
    output_path=output_ply,
    contact_threshold=0.08,
    num_sample_points=50000
    
)

Output saved to temp.ply


In [12]:
original = pymesh.load_mesh("data/shrink.ply")
final = pymesh.load_mesh("temp.ply")
print(pymesh.detect_self_intersection(original))
print(pymesh.detect_self_intersection(final))

[[ 465  844]
 [ 465 1199]
 [ 465  830]
 [ 465 1156]
 [ 828  465]
 [1154  465]
 [1211  465]
 [ 864  465]
 [1232  465]
 [ 825  465]
 [1194  465]]
[]


In [13]:
visualize_two_meshes(original, final)

Widget(value='<iframe src="http://localhost:37889/index.html?ui=P_0x7f1e26249270_2&reconnect=auto" class="pyvi…

None

In [5]:
def pymesh_to_trimesh(mesh):
    return trimesh.Trimesh(vertices=mesh.vertices, faces=mesh.faces)

def trimesh_to_pymesh(mesh):
    return pymesh.form_mesh(vertices=mesh.vertices, faces=mesh.faces)

In [6]:
tri_original = pymesh_to_trimesh(original)
tri_final = pymesh_to_trimesh(final)

In [7]:
original_submeshes = tri_original.split(only_watertight=True)
original_inner = sorted(original_submeshes, key=lambda m: m.volume)[0]
final_submeshes = tri_final.split(only_watertight=True)
final_inner = sorted(final_submeshes, key=lambda m: m.volume)[0]

In [8]:
evaluation(original_inner, final_inner)

+-----------------------------------+-----------+-----------+
| Metric                            |    Before |     After |
+===================================+===========+===========+
| Number of vertices                | 368       | 368       |
+-----------------------------------+-----------+-----------+
| Number of faces                   | 732       | 732       |
+-----------------------------------+-----------+-----------+
| Number of intersecting face pairs |   0       |   0       |
+-----------------------------------+-----------+-----------+
| Volume                            |   1.27715 |   1.23444 |
+-----------------------------------+-----------+-----------+
| Area                              |   5.7148  |   5.59863 |
+-----------------------------------+-----------+-----------+
| Intact vertices (%)               | nan       |  80.7065  |
+-----------------------------------+-----------+-----------+


In [9]:
original_inner_py = trimesh_to_pymesh(original_inner)
final_inner_py = trimesh_to_pymesh(final_inner)

In [10]:
visualize_two_meshes(original_inner_py, final_inner_py)

Widget(value='<iframe src="http://localhost:37889/index.html?ui=P_0x7f1e48790a00_1&reconnect=auto" class="pyvi…

None